In [2]:
#Importar as bibliotecas necessárias

import pandas as pd
#print('Pandas version: ', pd.__version__)

import numpy as np
#print('Numpy version: ', np.__version__)

import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error

import itertools

from sklearn.metrics import silhouette_samples, silhouette_score

In [3]:
# Importar o dataset de filmes
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Importar o dataset de avaliações
ratings = pd.read_csv('ratings.csv', usecols = ['userId', 'movieId','rating'])
print('Shape of ratings dataset is: ',ratings.shape, '\n')
print('Max values in dataset are \n',ratings.max(), '\n')
print('Min values in dataset are \n',ratings.min(), '\n') 
ratings.head()

Shape of ratings dataset is:  (25000095, 3) 

Max values in dataset are 
 userId     162541.0
movieId    209171.0
rating          5.0
dtype: float64 

Min values in dataset are 
 userId     1.0
movieId    1.0
rating     0.5
dtype: float64 



,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [5]:
ratings_title = pd.merge(ratings, movies[['movieId', 'title']], on = 'movieId')
print('Shape of ratings tittle dataset is: ',ratings_title.shape, '\n')
ratings_title.head()

Shape of ratings tittle dataset is:  (25000095, 4) 



,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,3,296,5.0,Pulp Fiction (1994)
2,4,296,4.0,Pulp Fiction (1994)
3,5,296,4.0,Pulp Fiction (1994)
4,7,296,4.0,Pulp Fiction (1994)


In [6]:
user_movie_ratings = pd.pivot_table(ratings_title[:10000000], index = 'userId', columns = 'title', values = 'rating')

In [7]:
def get_most_rated_movies(user_movie_ratings, max_number_of_movies):
    # 1- Count
    user_movie_ratings = user_movie_ratings.append(user_movie_ratings.count(), ignore_index=True)
    # 2- sort
    user_movie_ratings_sorted = user_movie_ratings.sort_values(len(user_movie_ratings)-1, axis=1, ascending=False)
    user_movie_ratings_sorted = user_movie_ratings_sorted.drop(user_movie_ratings_sorted.tail(1).index)
    # 3- slice
    most_rated_movies = user_movie_ratings_sorted.iloc[:, :max_number_of_movies]
    return most_rated_movies

In [8]:
user_movie_ratings =  pd.pivot_table(ratings_title[:10000000], index = 'userId', columns = 'title', values = 'rating')
most_rated_movies_1k = get_most_rated_movies(user_movie_ratings, 1000)

In [9]:
print (most_rated_movies_1k.shape)
most_rated_movies_1k.head()

(162452, 949)


title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Schindler's List (1993),Braveheart (1995),Fight Club (1999),...,The Trust (2016),Death Note Rewrite: Genshisuru Kami (2007),Hit the Bank (Vabank) (1981),Teddy Bear (Mis) (1981),Ghost in the Shell: The New Movie (2015),Ford v. Ferrari (2019),Swann in Love (Un amour de Swann) (1984),"Miracle of Marcelino, The (Marcelino pan y vino) (1955)",Fateful Findings (2013),Alien Planet (2005)
0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,5.0,NaN,NaN,3.5,1.0,NaN,NaN
1,4.5,5.0,NaN,NaN,5.0,5.0,2.0,5.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,4.0,5.0,4.0,4.0,4.0,2.0,4.0,NaN,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4.0,NaN,4.5,3.5,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN,2.0,3.0
4,4.0,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
sparse_ratings = csr_matrix(most_rated_movies_1k)
sparse_ratings

<162452x949 sparse matrix of type '<class 'numpy.float64'>'
	with 154166948 stored elements in Compressed Sparse Row format>

In [11]:
sparseratings = pd.DataFrame.sparse.from_spmatrix(sparse_ratings)
sparseratings = sparseratings.fillna(0)
sparseratings = sparseratings.sparse.to_coo()
sparseratings

<162452x949 sparse matrix of type '<class 'numpy.float64'>'
	with 154166948 stored elements in COOrdinate format>

In [ ]:
k_means = KMeans(n_clusters = 150, algorithm = 'full',max_iter = 100)

predictions = k_means.fit_predict(sparseratings)

clustered = pd.concat([most_rated_movies_1k.reset_index(), pd.DataFrame({'group':predictions})], axis=1)

In [14]:
cluster_number = 7
cluster = clustered[clustered.group == cluster_number].drop(['index', 'group'], axis=1)

In [14]:
cluster.fillna('').head()

NameError: name 'cluster' is not defined

In [16]:
movie_name = "Pulp Fiction (1994)"
cluster[movie_name].mean()

4.324015247776366

In [17]:
cluster.mean().head(20)

Forrest Gump (1994)                                                               3.916494
Shawshank Redemption, The (1994)                                                  4.483670
Pulp Fiction (1994)                                                               4.324015
Silence of the Lambs, The (1991)                                                  4.316734
Matrix, The (1999)                                                                4.228157
Star Wars: Episode IV - A New Hope (1977)                                         4.447904
Jurassic Park (1993)                                                              3.702624
Schindler's List (1993)                                                           4.408115
Braveheart (1995)                                                                 4.105357
Fight Club (1999)                                                                 4.096488
Terminator 2: Judgment Day (1991)                                                 4.028285

In [18]:
user_id = 71

user_2_ratings  = cluster.loc[user_id, :]

user_2_unrated_movies =  user_2_ratings[user_2_ratings.isnull()]


avg_ratings = pd.concat([user_2_unrated_movies, cluster.mean()], axis=1, join='inner').loc[:,0]

avg_ratings.sort_values(ascending=False)[:20]

Girl Who Kicked the Hornet's Nest, The (Luftslottet som sprängdes) (2009)    4.625000
Chef (2014)                                                                  4.500000
Resident Evil: Extinction (2007)                                             4.500000
Seven Samurai (Shichinin no samurai) (1954)                                  4.449607
Wallace & Gromit: A Close Shave (1995)                                       4.418211
Rashomon (Rashômon) (1950)                                                   4.305556
Memento (2000)                                                               4.283656
Virgin Spring, The (Jungfrukällan) (1960)                                    4.274194
Ran (1985)                                                                   4.267327
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)                         4.265823
Lord of the Rings: The Fellowship of the Ring, The (2001)                    4.260524
Spirited Away (Sen to Chihiro no kamikakushi) (2001)  

In [22]:
frame = pd.DataFrame(sparse_ratings)
frame['cluster'] = predictions
frame['cluster'].value_counts()

13    43879
4     19872
8     16621
2     16115
5     11035
12     9271
3      8739
0      6846
6      6565
11     6531
7      4590
1      4579
9      3409
14     2566
10     1834
Name: cluster, dtype: int64

In [19]:
k_means.inertia_

92388102.20344439

In [21]:
k_means.get_params()

{'algorithm': 'full',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 100,
 'n_clusters': 15,
 'n_init': 10,
 'n_jobs': 'deprecated',
 'precompute_distances': 'deprecated',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}